In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from functools import reduce

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]

In [3]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term3 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) for j in range(n)])
        term4 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])

        hamiltonian += term1 @ term2 + term3 @ term4

    return hamiltonian

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr

In [4]:
def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2
    return Hp
        
Hp = generate_Hp(n, m, d, g)

In [5]:
import sys
sys.path.append('../../')
import zlibrary.linear_system as ls
Cons = np.array([
                [-1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
                [0, -1, 0, 1, 0, 0, 0, 1, 0, 0],
                [0, -1, 0, 0, 0, 1, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
u = ls.find_basic_solution(Cons)
v = np.where(u == 1, 1, 0)
w = np.where(u == -1, 1, 0)
Hd1 = calculate_hamiltonian(v, w)
Cons = np.array([
                [-1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                [0, -1, 0, 0, 0, 1, 0, 0, 0, 1],
                [0, -1, 0, 1, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
u = ls.find_basic_solution(Cons)
v = np.where(u == 1, 1, 0)
w = np.where(u == -1, 1, 0)
Hd2 = calculate_hamiltonian(v, w)
Cons = np.array([
                [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]])
u = ls.find_basic_solution(Cons)
v = np.where(u == 1, 1, 0)
w = np.where(u == -1, 1, 0)
Hd3 = calculate_hamiltonian(v, w)
print(Hd1)
print(Hd2)
print(Hd3)

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


In [6]:
from scipy.linalg import expm
def build_circ(params):
  depth = len(params) // 6
  qc = QuantumCircuit(num_qubits)
  beta1 = params[0:depth]
  beta2 = params[depth:depth*2]
  beta3 = params[depth*2:depth*3]
  gamma1 = params[depth*3:depth*4]
  gamma2 = params[depth*4:depth*5]
  gamma3 = params[depth*5:depth*6]
  for i in [4,6,8]:
    qc.x(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * beta1[dp] * Hp), range(num_qubits))
    qc.unitary(expm(-1j * gamma1[dp] * Hd1), range(num_qubits))
    qc.unitary(expm(-1j * beta2[dp] * Hp), range(num_qubits))
    qc.unitary(expm(-1j * gamma2[dp] * Hd2), range(num_qubits))
    qc.unitary(expm(-1j * beta3[dp] * Hp), range(num_qubits))
    qc.unitary(expm(-1j * gamma3[dp] * Hd3), range(num_qubits))
  qc.measure_all()
  return qc

In [7]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [8]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(par):
  global iteration_count
  iteration_count = 0
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 2 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations =0

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, par, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.2f}%".format(counts[x] / shots * 100), cost_function(x))

In [9]:
for dep in range(1,10):
  print(f'depth == {dep}')
  test(np.full(dep * 6, np.pi/3))
  print()

depth == 1


Iteration 2, Result: 5.0085
Final Result:  message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: 5.0085
       x: [ 1.047e+00  1.047e+00  1.047e+00  1.047e+00  1.047e+00
            1.047e+00]
    nfev: 1
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1100001111 4.20% 3
1101000111 2.32% 5
1111000011 2.31% 6
1110001011 2.24% 4
1110000111 2.12% 4
1100011111 1.87% 5
1101001111 1.81% 5
1011100110 1.65% 6
1000101111 1.46% 3
1001100111 1.36% 5
1101010111 1.35% 7
1010101110 1.25% 4
1101000011 1.22% 5
1011000110 1.21% 5
1011010110 1.18% 7
1111000111 1.10% 6
1100000111 0.95% 3
1011100111 0.92% 6
1111010011 0.92% 8
1010101111 0.90% 4

depth == 2
Iteration 2, Result: 4.4555
Final Result:  message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: 4.4555
       x: [ 1.047e+00  1.047e+00  1.047e+00  1.04